In [ ]:
import pickle
import nltk
import re
import pandas as pd
import numpy as np
from urllib.request import urlopen
from pathlib import Path
import re
from bs4 import BeautifulSoup
# from tika import parser
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
# from transformers import pipeline
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


# New Section

In [ ]:
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger',
               'stopwords', 'vader_lexicon'])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
ENGLISH = 'english'
FEATURES = 'features'
TEXT_PIPELINE = 'text_pipeline'
VECT = 'vect'
TFIDF = 'tfidf'
TOKENIZED = 'tokenized'
LETTER_TEXT = 'letter_text'
COSINE = 'cosine'
EUCLIDEAN = 'euclidean'
WORD2VEC = 'word2vec'
DOC2VECT = 'doc2vect'

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


# New Section

In [ ]:
import os
folder_path = '/content/drive/MyDrive/NLP-CS510/20news-bydate-train/misc.forsale/'
# arr = os.listdir()
# arr = os.listdir(folder_path)
# print(arr)
# for i in arr:
#   files = os.listdir(folder_path)
# content = open('/content/drive/MyDrive/NLP-CS510/20news-bydate-train/misc.forsale/76562', 'r').read()
# print(content)

In [ ]:
import os
from pathlib import Path
def get_text_from_pdf(path):
  txt = open(path, 'rb')
  data = txt.read()
  txt.close()
  return data

In [ ]:
def get_letters_corpus_dict(letters_pdf_path):
    letters_dict = dict()
    letters_years = [field for field in os.listdir(folder_path)]
    for year in letters_years:
      path = Path(letters_pdf_path).joinpath(year)
      # print("year:"+year)
      letter_corpus = get_text_from_pdf(str(path))
      # print("path: "+ str(path))
      letters_dict[year] = letter_corpus
    return letters_dict

input_dict = get_letters_corpus_dict(folder_path)
file_to_write = open("letters_dict.pickle", "wb")

pickle.dump(input_dict, file_to_write)

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
def tokenize(text):
  ###add utf encoding remove 'b' word
    text = str(text).lower()
    text = re.sub("[^a-zA-Z0-9]", " ", text)
    tokens = word_tokenize(text)
    tokens = [w for w in tokens if w not in stopwords.words(ENGLISH)]

    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()

    clean_tokens_list = []
    for tok in tokens:
        lemmatizer_tok = lemmatizer.lemmatize(tok).strip()
        clean_tok = stemmer.stem(lemmatizer_tok)
        clean_tokens_list.append(clean_tok)

    return clean_tokens_list

In [ ]:
def get_letters_df():
  with open('letters_dict.pickle', 'rb') as handle:
    letters_dict = pickle.load(handle)
    letters_df = pd.DataFrame(input_dict, index=[LETTER_TEXT]).T
    print(letters_df.count())
    letters_df[TOKENIZED] = letters_df[LETTER_TEXT].apply(tokenize)
    # letters_df = letters_df.apply(tokenize)
  return letters_df

input_df = get_letters_df()
print(input_df[TOKENIZED].loc["74776"])

letter_text    585
dtype: int64
['b', 'wil', 'chin', 'launchpad', 'unc', 'edu', 'wilson', 'chin', 'nsubject', 'datman', 'sale', 'nnntp', 'post', 'host', 'lambada', 'oit', 'unc', 'edu', 'norgan', 'univers', 'north', 'carolina', 'extend', 'bulletin', 'board', 'servic', 'nline', '19', 'n', 'nin', 'articl', 'bdavisc4yum3', 'k3j', 'netcom', 'com', 'bdavi', 'netcom', 'com', 'bryan', 'davi', 'nwrite', 'n', 'soni', 'tcd', 'd3', 'dat', 'walkman', 'sale', 'hardli', 'use', 'still', 'n', 'warrante', 'come', 'optic', 'digit', 'input', 'output', 'well', 'standard', 'n', 'rca', 'analog', 'cabl', 'includ', 'rechar', 'batteri', 'charger', 'ac', 'adaptor', 'n', '650', '00', 'best', 'offer', 'n', 'njust', 'inform', 'reader', 'newsrc', 'soni', 'introduc', 'new', 'datman', 'nthe', 'tcd', 'd7', 'smaller', 'le', 'expens', 'd3', 'also', 'd7', 'nha', 'coaxial', 'jack', 'd3', 'lack', 'oh', 'd3', 'alway', 'navail', 'mail', 'order', 'hous', 'le', '700', 'dollar', 'new', 'nthe', 'new', 'model', 'come', 'suspect', 

In [ ]:
import gensim
tagged_data = [gensim.models.doc2vec.TaggedDocument(d, [i]) for i, d in enumerate((input_df[TOKENIZED]))]
tagged_data

[TaggedDocument(words=['b', 'bpang', 'tuba', 'calpoli', 'edu', 'brennan', 'lawrenc', 'pang', 'nsubject', 'hard', 'drive', 'etc', 'sale', 'norgan', 'california', 'polytechn', 'state', 'univers', 'san', 'lui', 'obispo', 'nline', '35', 'n', 'nhard', 'drive', 'drive', 'includ', 'manual', 'warranti', 'n', 'nfujitsu', '1', '2', 'gigabyt', 'scsi', 'hard', 'drive', 'n', '1000', 'n', 'nquantum', '240', 'meg', 'scsi', 'prodriv', 'n', '350', 'n', 'nfujitsu', '90', 'meg', 'scsi', 'hard', 'drive', 'n', '175', 'n', 'nappl', 'zero', 'footprint', 'hard', 'drive', 'case', 'power', 'suppli', 'n', '75', 'n', 'nacceller', 'video', 'n', 'nappl', '8', '24gc', 'acceller', 'graphic', 'card', 'softwar', 'init', 'nmillion', 'color', 'fast', 'n', '700', 'n', 'nhardcopi', 'n', 'nappl', 'imagewrit', 'ii', 'printer', 'spare', 'print', 'head', 'cheap', 'n', '300', 'nfax', 'machin', '9600', 'baud', 'leather', 'travel', 'case', 'mani', 'accessori', 'n', '200', 'n', 'nto', 'discu', 'purchas', 'item', 'call', 'john', '4

In [ ]:
## Train doc2vec model
model = gensim.models.doc2vec.Doc2Vec(tagged_data, vector_size=20, window=2, min_count=1, workers=4, epochs = 100)
# Save trained doc2vec model
model.save("test_doc2vec.model")
## Load saved doc2vec model
model= gensim.models.doc2vec.Doc2Vec.load("test_doc2vec.model")
## Print model vocabulary
print(model.wv.vocab)

{'b': <gensim.models.keyedvectors.Vocab object at 0x7fd51648f0d0>, 'bpang': <gensim.models.keyedvectors.Vocab object at 0x7fd51648f250>, 'tuba': <gensim.models.keyedvectors.Vocab object at 0x7fd51648f210>, 'calpoli': <gensim.models.keyedvectors.Vocab object at 0x7fd51648f310>, 'edu': <gensim.models.keyedvectors.Vocab object at 0x7fd51648f390>, 'brennan': <gensim.models.keyedvectors.Vocab object at 0x7fd51648f410>, 'lawrenc': <gensim.models.keyedvectors.Vocab object at 0x7fd51648f490>, 'pang': <gensim.models.keyedvectors.Vocab object at 0x7fd51648f550>, 'nsubject': <gensim.models.keyedvectors.Vocab object at 0x7fd51648f510>, 'hard': <gensim.models.keyedvectors.Vocab object at 0x7fd51648f610>, 'drive': <gensim.models.keyedvectors.Vocab object at 0x7fd51648f690>, 'etc': <gensim.models.keyedvectors.Vocab object at 0x7fd51648f710>, 'sale': <gensim.models.keyedvectors.Vocab object at 0x7fd51648f790>, 'norgan': <gensim.models.keyedvectors.Vocab object at 0x7fd51648f850>, 'california': <gensim

In [ ]:
test_doc = word_tokenize(str(input_df.loc["74776"]))
model.docvecs.most_similar(positive=[model.infer_vector(test_doc)],topn=5)
print(input_df.iloc[558])
print(input_df.iloc[514])
print(input_df.iloc[17])
print(input_df.iloc[269])
print(input_df.iloc[120])

letter_text    b'From: kpeterso@nyx.cs.du.edu (Kirk Peterson)...
tokenized      [b, kpeterso, nyx, c, du, edu, kirk, peterson,...
Name: 76394, dtype: object
letter_text    b'From: jingyao@rainier.eng.ohio-state.edu (Ji...
tokenized      [b, jingyao, rainier, eng, ohio, state, edu, j...
Name: 76157, dtype: object
letter_text    b'From: HO@kcgl1.eng.ohio-state.edu (Francis H...
tokenized      [b, ho, kcgl1, eng, ohio, state, edu, franci, ...
Name: 76023, dtype: object
letter_text    b"From: Michelle Zumbo <mz10+@andrew.cmu.edu>\...
tokenized      [b, michel, zumbo, mz10, andrew, cmu, edu, nsu...
Name: 76038, dtype: object
letter_text    b'From: dtmedin@catbyte.b30.ingr.com (Dave Med...
tokenized      [b, dtmedin, catbyt, b30, ingr, com, dave, med...
Name: 75864, dtype: object
